## Sample game

Play with pretrained agent yourself!

In [ ]:
import sys
import time
from IPython.display import clear_output

if '..' not in sys.path: sys.path.append('..')
from src.utils.common import *
from src.dnd.actions import *
from src.dnd.units import *
from src.dnd.game_utils import *
from src.dnd.game_board import DnDBoard
from src.agent.agent import DnDAgent
from src.agent.agent_utils import get_states

In [ ]:
def create_game():
    global game, color_map # hell yeah globals

    game = DnDBoard((5, 5))
    ally = GenericSoldier("Ally soldier", attack_damage=25)
    ally2 = GenericSoldier("Ally ranger", attack_damage=25, health=50, range=4)
    enemy = GenericSoldier("Enemy soldier", attack_damage=25)
    enemy2 = GenericSoldier("Enemy ranger", attack_damage=25, health=50, range=4)

    place_unit_randomly(game, ally, 0)
    place_unit_randomly(game, enemy, 1)
    place_unit_randomly(game, ally2, 0)
    place_unit_randomly(game, enemy2, 1)

    game.initialize_game()

    color_map = {
        ally: "Green",
        enemy: "Red",
        ally2: "Blue",
        enemy2: "Purple"
    }

load agent:

In [ ]:
checkpoint_path = '../rnd/2v2-0/trained-agents/agent-gen30-11.1i-600000'
agent = DnDAgent.load_agent(checkpoint_path, strip=True, epsilon=0)

Start new game:

In [ ]:
create_game()

In [ ]:
game_over = False

print_game(game, color_map)
print('Press Enter to start')
_ = input()

while not game_over:
    if game.current_player_id == 1:
        # clear_output()
        _, _, new_coords, action = get_states(game, agent)
        _, game_over = take_turn(game, new_coords, action, color_map, True)
        continue
    
    while True:
        print("It's your turn. Enter the desired coordinates to move to `y x`:", end='')
        new_coords = tuple([int(x) for x in input().split()])
        print(new_coords)

        print('Enter the coordinates of unit you want to attack `y x`, or press Enter to skip attack:', end='')
        attack_input = input()
        action = None
        target_pos = (-1, -1)
        if attack_input != '':
            unit = game.current_unit
            target_pos = tuple([int(x) for x in attack_input.split()])
            target_unit = game.board[target_pos]
            action = ActionInstance(unit.actions[0], source_unit=unit, target_unit=target_unit)
        print(target_pos)

        try:
            _, game_over = game.take_turn(new_coords, action)
        except Exception as e:
            print(e)
            continue
        
        print_game(game, color_map)
        break